In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
#database connection settings
import psycopg2
import pandas as pd

db_name = "traviato_development"
db_host = "localhost"
db_port = "5432"
db_user = "lievgarcia"
db_pwd = "traviato81"

conn = psycopg2.connect(database=db_name, user=db_user, password=db_pwd, host=db_host, port=db_port)

cursor = conn.cursor()  
query = "select id, name, url from composers c ORDER BY name"
cursor.execute(query)
c_df = pd.DataFrame(cursor.fetchall(), columns=['id', 'name', 'url'])

In [ ]:
# c_df[c_df['url'].isna()].head()

In [ ]:
#################### ENHANCE WITH WIKIDATA LINKS ####################
import requests
import urllib.parse

def url_text(text):
    return urllib.parse.quote_plus(text)

endpoint = "https://www.wikidata.org/w/api.php"
params = "?language=en&format=json"
action = "&action=wbsearchentities&search="

print(c_df.shape)
c_df = c_df[c_df['url'].isna()]
print(c_df.shape)

for idx,row in c_df.iterrows():
    query  = url_text(row['name'])
    itemsearch = requests.get(endpoint + params + action + query)
    itemsearch = itemsearch.json()
    try:
        uri = itemsearch['search'][0]['id']
        url = itemsearch['search'][0]['concepturi']
        
        update = "UPDATE COMPOSERS "
        values = "SET URI = '" + uri + "', URL = '" +  url + "' "
        where  = "WHERE id = " + str(row['id']) + "; "
        commit = "COMMIT;"
        print(update + values + where + commit)    
        cursor.execute(update + values + where + commit)        
    except:
        None
####################################################################

In [4]:
#################### UPDATE COMPOSER ATTRIBUTES ####################

import psycopg2
import pandas as pd

db_name = "traviato_development"
db_host = "localhost"
db_port = "5432"
db_user = "lievgarcia"
db_pwd = "traviato81"

conn = psycopg2.connect(database=db_name, user=db_user, password=db_pwd, host=db_host, port=db_port)

attribute = 'premiere_date'
attribute_id = 'P1249' #, 1249

cursor = conn.cursor()  
query = "select id, name, uri from works c WHERE " + attribute + " IS NULL ORDER BY name"
cursor.execute(query)
c_df = pd.DataFrame(cursor.fetchall(), columns=['id', 'name', 'uri'])
c_df = c_df[~c_df['uri'].isna()]
print(c_df.shape)

from wikidata.client import Client
client = Client()

for idx,row in c_df.iterrows():
    
    try:    
        
        entity    = client.get(row['uri'], load=True)
        name      = str(entity.label)
        attr_prop = client.get(attribute_id)
        try:            
            #for strings
    #         attr = str(entity[attr_prop].label).title()
            #for dates
            attr = str(entity[attr_prop].year)
        except:
            attr = entity.attributes['claims'][attribute_id][0]['mainsnak']['datavalue']['value']['time'][1:5]            
        
#         entity    = client.get(row['uri'], load=True)
#         attr_id   = entity.id
#         attr_prop = client.get(attribute_id)
#         attr      = str(entity[attr_prop].id)
    
#         entity    = client.get(attr, load=True)
#         name      = str(entity.label)
#         attr_prop = client.get('P17')     
#         attr      = str(entity[attr_prop].label).title()

        update = "UPDATE WORKS "
        values = "SET " + attribute + " = '" + attr.replace("'", "''") + "' "
        where  = "WHERE id = " + str(row['id']) + "; "
        commit = "COMMIT;"
        print(update + values + where + commit)    
        cursor.execute(update + values + where + commit)    
    except:
        None

(6640, 3)


In [36]:
from bs4 import BeautifulSoup
import requests, urllib
from datetime import datetime

headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }
base_url = 'http://operadata.stanford.edu/?utf8=%E2%9C%93&utf8=%E2%9C%93&search_field=title&'

#################### UPDATE COMPOSER ATTRIBUTES ####################
import psycopg2
import pandas as pd

db_name = "traviato_development"
db_host = "localhost"
db_port = "5432"
db_user = "lievgarcia"
db_pwd = "traviato81"

conn = psycopg2.connect(database=db_name, user=db_user, password=db_pwd, host=db_host, port=db_port)
#######################################################################

cursor = conn.cursor()  
query = "select id, name, uri from works c WHERE " + attribute + " IS NULL ORDER BY name"
cursor.execute(query)
c_df = pd.DataFrame(cursor.fetchall(), columns=['id', 'name', 'uri'])
print(c_df.shape)

for idx,row in c_df.iterrows():
    
    if idx%250==0:
        print(idx)
        
    params = {
        'q':row['name']
    }
    
    try:
        params = urllib.parse.urlencode(params)
        params = params.replace('+', '-')
        url = base_url + params
#         print(url)
    except:
        None
        
    r  = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "html5lib")
    results = soup.find(attrs={'class': 'document-metadata dl-horizontal dl-invert'})
    match=False
    if results:
        for item in list(zip(results.find_all("dt"),results.find_all("dd"))):
            if item[0].get_text()=='Year:':
                year = item[1].get_text()
            if item[0].get_text()=='Title:' and item[1].get_text().lower()==row['name'].lower():
                match=True
    
    if match==True:
                update = "UPDATE WORKS "
                values = "SET premiere_date = '" + str(year) + "' "
                where  = "WHERE id = " + str(row['id']) + "; "
                commit = "COMMIT;"
                print(update + values + where + commit)    
                cursor.execute(update + values + where + commit)    
                

(4352, 3)
0
250
500
750
1000
1250
1500
1750
2000
2250
2500
2750
UPDATE WORKS SET premiere_date = '1783' WHERE id = 1741; COMMIT;
UPDATE WORKS SET premiere_date = '1783' WHERE id = 16892; COMMIT;
UPDATE WORKS SET premiere_date = '1783' WHERE id = 1176; COMMIT;
UPDATE WORKS SET premiere_date = '1783' WHERE id = 1514; COMMIT;
UPDATE WORKS SET premiere_date = '1783' WHERE id = 3197; COMMIT;
UPDATE WORKS SET premiere_date = '1689' WHERE id = 16893; COMMIT;
UPDATE WORKS SET premiere_date = '1959' WHERE id = 8617; COMMIT;
UPDATE WORKS SET premiere_date = '2009' WHERE id = 21760; COMMIT;
UPDATE WORKS SET premiere_date = '2013' WHERE id = 22834; COMMIT;
UPDATE WORKS SET premiere_date = '1971' WHERE id = 22883; COMMIT;
UPDATE WORKS SET premiere_date = '1971' WHERE id = 21720; COMMIT;
UPDATE WORKS SET premiere_date = '1949' WHERE id = 20513; COMMIT;
UPDATE WORKS SET premiere_date = '1949' WHERE id = 21989; COMMIT;
UPDATE WORKS SET premiere_date = '2012' WHERE id = 2917; COMMIT;
UPDATE WORKS SET p

UPDATE WORKS SET premiere_date = '1863' WHERE id = 3890; COMMIT;
UPDATE WORKS SET premiere_date = '2016' WHERE id = 21960; COMMIT;
UPDATE WORKS SET premiere_date = '1939' WHERE id = 940; COMMIT;
UPDATE WORKS SET premiere_date = '2017' WHERE id = 22458; COMMIT;
UPDATE WORKS SET premiere_date = '2002' WHERE id = 22164; COMMIT;
UPDATE WORKS SET premiere_date = '1900' WHERE id = 23259; COMMIT;
UPDATE WORKS SET premiere_date = '1953' WHERE id = 2792; COMMIT;
UPDATE WORKS SET premiere_date = '1879' WHERE id = 9018; COMMIT;
UPDATE WORKS SET premiere_date = '1903' WHERE id = 9022; COMMIT;
UPDATE WORKS SET premiere_date = '1978' WHERE id = 9023; COMMIT;
UPDATE WORKS SET premiere_date = '2007' WHERE id = 9024; COMMIT;
UPDATE WORKS SET premiere_date = '1953' WHERE id = 2663; COMMIT;
UPDATE WORKS SET premiere_date = '1999' WHERE id = 9029; COMMIT;
UPDATE WORKS SET premiere_date = '1903' WHERE id = 17494; COMMIT;
UPDATE WORKS SET premiere_date = '1903' WHERE id = 17495; COMMIT;
UPDATE WORKS SET pre

UPDATE WORKS SET premiere_date = '2015' WHERE id = 22490; COMMIT;
UPDATE WORKS SET premiere_date = '1911' WHERE id = 17964; COMMIT;
UPDATE WORKS SET premiere_date = '1958' WHERE id = 9384; COMMIT;
UPDATE WORKS SET premiere_date = '2015' WHERE id = 22857; COMMIT;
UPDATE WORKS SET premiere_date = '1903' WHERE id = 17975; COMMIT;
UPDATE WORKS SET premiere_date = '2014' WHERE id = 22710; COMMIT;
UPDATE WORKS SET premiere_date = '1980' WHERE id = 737; COMMIT;
UPDATE WORKS SET premiere_date = '1980' WHERE id = 855; COMMIT;
UPDATE WORKS SET premiere_date = '2010' WHERE id = 22753; COMMIT;
UPDATE WORKS SET premiere_date = '2012' WHERE id = 21411; COMMIT;
UPDATE WORKS SET premiere_date = '1966' WHERE id = 21372; COMMIT;
UPDATE WORKS SET premiere_date = '2016' WHERE id = 17999; COMMIT;
UPDATE WORKS SET premiere_date = '2016' WHERE id = 22498; COMMIT;
UPDATE WORKS SET premiere_date = '1982' WHERE id = 21374; COMMIT;
UPDATE WORKS SET premiere_date = '1894' WHERE id = 18001; COMMIT;
UPDATE WORKS SE

UPDATE WORKS SET premiere_date = '1970' WHERE id = 21772; COMMIT;
UPDATE WORKS SET premiere_date = '1786' WHERE id = 2463; COMMIT;
UPDATE WORKS SET premiere_date = '1967' WHERE id = 3535; COMMIT;
UPDATE WORKS SET premiere_date = '2014' WHERE id = 22465; COMMIT;
UPDATE WORKS SET premiere_date = '1957' WHERE id = 21532; COMMIT;
UPDATE WORKS SET premiere_date = '1787' WHERE id = 4836; COMMIT;
UPDATE WORKS SET premiere_date = '2010' WHERE id = 22552; COMMIT;
UPDATE WORKS SET premiere_date = '1969' WHERE id = 23091; COMMIT;
UPDATE WORKS SET premiere_date = '1969' WHERE id = 18442; COMMIT;
UPDATE WORKS SET premiere_date = '2015' WHERE id = 23061; COMMIT;
UPDATE WORKS SET premiere_date = '1898' WHERE id = 18447; COMMIT;
UPDATE WORKS SET premiere_date = '2017' WHERE id = 22871; COMMIT;
UPDATE WORKS SET premiere_date = '2011' WHERE id = 22316; COMMIT;
UPDATE WORKS SET premiere_date = '1906' WHERE id = 23224; COMMIT;
UPDATE WORKS SET premiere_date = '1962' WHERE id = 1116; COMMIT;
UPDATE WORKS S